In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip uninstall -y torch torchdata torchvision torchtext torchaudio fastai
!pip install portalocker
!pip install --pre torch torchdata -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
!pip install torchtext 

Found existing installation: torch 2.0.0+cu118
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/pyth

In [79]:
import math
import os
#import torch
#import torch.nn as nn
#import torch.nn.functional as F
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
import glob
import cv2
import os
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import tensorflow as tf
from mtcnn import MTCNN
from keras.applications.resnet import ResNet50
import mxnet as mx
from mxnet.gluon.model_zoo.vision import resnet34_v1,resnet34_v2
from keras.models import Sequential
from keras.layers import LSTM, Dense, MultiHeadAttention, Lambda, Conv1D, InputLayer
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from keras import metrics
from keras.applications.vgg16 import VGG16
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, LSTM, Attention, Flatten, GlobalAveragePooling1D, LayerNormalization, Dropout, Concatenate, Layer
from keras.models import Model
from tensorflow.keras.preprocessing.image import array_to_img
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.backend import variable

In [4]:
words = {1:"Begin", 2:"Choose", 3:"Connection", 4:"Navigation", 5:"Next",
         6:"Previous", 7:"Start", 8:"Stop", 9:"Hello", 10:"Web"}

phrases = {1:"Stop Navigation", 2:"Excuse se", 3:"I am sorry", 4:"Thank you", 5:"Good bye",
          6:"I love this game", 7:"Nice to meet you", 8:"You are welcome", 9:"How are you", 10:"Have a good time"}  

In [ ]:
parent_folder_path = "/content/gdrive/MyDrive/HW_CS7150_DL/ds1/"
persons = [os.path.join(parent_folder_path, name)+str("/") for name in os.listdir(parent_folder_path) if os.path.isdir(os.path.join(parent_folder_path, name))]
words_path = []
for i in persons:
  for name in os.listdir(i):
    w = os.path.join(i,name)
    if w[-5:] == "words":
      w += str("/")
      for name1 in os.listdir(w):
        w1 = os.path.join(w,name1) + str("/")
        for name2 in os.listdir(w1):
          w2 = os.path.join(w1,name2) + str("/")
          for name3 in os.listdir(w2):
            w3 = os.path.join(w2,name3)
            if w3[-13:-8] == "color":
              words_path.append(w3)
            
print(words_path)

['/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/01/color_003.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/01/color_002.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/01/color_001.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/01/color_004.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/01/color_006.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/01/color_005.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/09/color_001.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/09/color_006.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/09/color_005.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/09/color_002.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/09/color_007.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/09/color_004.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/09/09/color_003.jpg', '/content/gdrive/MyDrive/HW_CS7150_DL

In [ ]:
i=words_path[0]
print(f"cropped_{i[41:44]}_{i[45:49]}_{i[51:53]}_instance_{i[54:56]}_frame_{i[-5:-4]}.jpg")
print(len(words_path))

cropped_F01_word_09_instance_01_frame_3.jpg
4915


In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow
import tensorflow as tf
from mtcnn import MTCNN

# Load the MTCNN model
detector = MTCNN(min_face_size=75)

cnt1=0

save_path = "/content/gdrive/MyDrive/HW_CS7150_DL/cropped_v3/"
   # Load the image
img = cv2.imread("/content/gdrive/MyDrive/HW_CS7150_DL/ds1/F01/words/01/01/color_001.jpg")
   # Detect faces in the image
faces = detector.detect_faces(img)
   # Crop and save each detected face
cnt=0
for j, face in enumerate(faces):
  x, y, w, h = face["box"]
  cropped_img = img[y:y+h, x:x+w]

    # Crop the bounding box area and save it as a new image
  box_size=65
  x1 = x + w // 2 - box_size // 2  # calculate top-left corner x-coordinate
  y1 = y + h // 2 + 12    #box_size // 25  # calculate top-left corner y-coordinate
  x2 = x1 + box_size-7            # calculate bottom-right corner x-coordinate
  y2 = y1 + 25   #box_size//2             # calculate bottom-right corner y-coordinate
  cropped_box = img[y1:y2, x1:x2]
  cv2.imwrite(os.path.join(save_path, f"cropped.jpg"), cropped_box)
    

1/1 [==============================] - 0s 124ms/step


In [ ]:
# Load the MTCNN model
detector = MTCNN(min_face_size=75)
save_path = '/content/gdrive/MyDrive/HW_CS7150_DL/cropped_v3/'
for k in range(len(words_path)):
   i = words_path[k]
   # Load the image
   img = cv2.imread(words_path[k])
   # Detect faces in the image
   faces = detector.detect_faces(img)
   # Crop and save each detected face
   cnt=0
   for j, face in enumerate(faces):
     x, y, w, h = face["box"]
     cropped_img = img[y:y+h, x:x+w]

    # Crop the bounding box area and save it as a new image
     box_size=65
     x1 = x + w // 2 - box_size // 2  # calculate top-left corner x-coordinate
     y1 = y + h // 2 + 12    #box_size // 25  # calculate top-left corner y-coordinate
     x2 = x1 + box_size-7            # calculate bottom-right corner x-coordinate
     y2 = y1 + 25   #box_size//2             # calculate bottom-right corner y-coordinate
     cropped_box = img[y1:y2, x1:x2]
     cv2.imwrite(os.path.join(save_path, f"cropped_{i[41:44]}_{i[45:49]}_{i[51:53]}_instance_{i[54:56]}_frame_{i[-5:-4]}.jpg"), cropped_box)
    
     

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 26ms/step


In [ ]:
# Import required libraries
import shutil

# Set the source and destination paths
source_folder_path = '/content/gdrive/MyDrive/HW_CS7150_DL/cropped'
destination_folder_path = '/content/gdrive/MyDrive/HW_CS7150_DL'

# Create a zip file of the source folder
shutil.make_archive('cropped', 'zip', source_folder_path)

# Download the zip file
from google.colab import files
files.download('/content/gdrive/MyDrive/HW_CS7150_DL/cropped.zip')

In [ ]:
from PIL import Image

# Specify the path of the folder containing the images
folder_path = '/content/gdrive/MyDrive/HW_CS7150_DL/cropped_new/'

img_sizes=[]
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
  with Image.open(os.path.join(folder_path, filename)) as img:
    width, height = img.size
    img_sizes.append(img.size)
    #print(f"Image: {filename}, Size: {width}x{height}")

#All the images are (25,58)

###Run from here

In [5]:
import os

folder_path = '/content/gdrive/MyDrive/HW_CS7150_DL/cropped_new/'

images = {}

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
  components = filename.split('_')
  person = components[1]
  word = components[3]
  instance = components[5]
  frame = components[7].split('.')[0]
        
        
  if person not in images:
    images[person] = {}
  if word not in images[person]:
    images[person][word] = {}
  if instance not in images[person][word]:
    images[person][word][instance] = {}
  images[person][word][instance][frame] = filename#os.path.join(folder_path, filename)
    
        

# Convert the dictionary to a nested list
images_list = []
for person in sorted(images.keys()):
    person_list = []
    for word in sorted(images[person].keys()):
        word_list = []
        for instance in sorted(images[person][word].keys()):
            instance_dict = images[person][word][instance]
            frame_list = [instance_dict[frame] for frame in sorted(instance_dict.keys())]
            word_list.append(frame_list)
        person_list.append(word_list)
    images_list.append(person_list)

images_list

[[[['cropped_F01_word_01_instance_01_frame_0.jpg',
    'cropped_F01_word_01_instance_01_frame_1.jpg',
    'cropped_F01_word_01_instance_01_frame_2.jpg',
    'cropped_F01_word_01_instance_01_frame_3.jpg',
    'cropped_F01_word_01_instance_01_frame_4.jpg',
    'cropped_F01_word_01_instance_01_frame_5.jpg',
    'cropped_F01_word_01_instance_01_frame_6.jpg',
    'cropped_F01_word_01_instance_01_frame_7.jpg',
    'cropped_F01_word_01_instance_01_frame_8.jpg',
    'cropped_F01_word_01_instance_01_frame_9.jpg'],
   ['cropped_F01_word_01_instance_03_frame_0.jpg',
    'cropped_F01_word_01_instance_03_frame_1.jpg',
    'cropped_F01_word_01_instance_03_frame_2.jpg',
    'cropped_F01_word_01_instance_03_frame_3.jpg',
    'cropped_F01_word_01_instance_03_frame_4.jpg',
    'cropped_F01_word_01_instance_03_frame_5.jpg',
    'cropped_F01_word_01_instance_03_frame_6.jpg',
    'cropped_F01_word_01_instance_03_frame_7.jpg',
    'cropped_F01_word_01_instance_03_frame_8.jpg',
    'cropped_F01_word_01_insta

In [6]:
img_names_in_sequence = []
frames_len_for_each_vid = []
for i in images_list:
  for j in i:
    for k in j:
      frames_len_for_each_vid.append(len(k))
      for imgs in k:
        img_names_in_sequence.append(imgs)
img_names_in_sequence[:20]

['cropped_F01_word_01_instance_01_frame_0.jpg',
 'cropped_F01_word_01_instance_01_frame_1.jpg',
 'cropped_F01_word_01_instance_01_frame_2.jpg',
 'cropped_F01_word_01_instance_01_frame_3.jpg',
 'cropped_F01_word_01_instance_01_frame_4.jpg',
 'cropped_F01_word_01_instance_01_frame_5.jpg',
 'cropped_F01_word_01_instance_01_frame_6.jpg',
 'cropped_F01_word_01_instance_01_frame_7.jpg',
 'cropped_F01_word_01_instance_01_frame_8.jpg',
 'cropped_F01_word_01_instance_01_frame_9.jpg',
 'cropped_F01_word_01_instance_03_frame_0.jpg',
 'cropped_F01_word_01_instance_03_frame_1.jpg',
 'cropped_F01_word_01_instance_03_frame_2.jpg',
 'cropped_F01_word_01_instance_03_frame_3.jpg',
 'cropped_F01_word_01_instance_03_frame_4.jpg',
 'cropped_F01_word_01_instance_03_frame_5.jpg',
 'cropped_F01_word_01_instance_03_frame_6.jpg',
 'cropped_F01_word_01_instance_03_frame_7.jpg',
 'cropped_F01_word_01_instance_03_frame_8.jpg',
 'cropped_F01_word_01_instance_03_frame_9.jpg']

In [7]:
#different grame lengths and their counts
lengths, counts = np.unique(np.array(frames_len_for_each_vid), return_counts=True)
max_len = max(lengths)
print("len:",lengths)
print("cnt:",counts)
print(sum(counts))

len: [ 1  2  4  5  6  7  8  9 10]
cnt: [  2   2   2   4  23  57  75  95 239]
499


In [8]:
image = cv2.imread('/content/gdrive/MyDrive/HW_CS7150_DL/cropped_new/cropped_F01_word_01_instance_01_frame_0.jpg')

# Convert the image to a NumPy array
image_array = np.array(image)
image_array.shape

(25, 58, 3)

In [9]:
folder = "/content/gdrive/MyDrive/HW_CS7150_DL/cropped_new/"
img_arr_in_sequence=[]
for i in images_list:
  arr1=[]
  for j in i:
    arr2=[]
    for k in j:
      arr3=[]
      for img in k:
        img_p = cv2.imread(folder+img)
        img_array = np.array(img_p)
        arr3.append(img_array)
      arr2.append(arr3)
    arr1.append(arr2)
  img_arr_in_sequence.append(arr1)


for i in img_arr_in_sequence:
  for j in i:
    pad_iter = 5-len(j)
    if pad_iter>0:
      j+= [[np.zeros((25,58,3))]*10]*pad_iter
    for k in j:
      pad_length = max_len-len(k)
      if pad_length>0:
        k+= ([np.zeros((25,58,3))]*pad_length)

pad_img_arr_in_sequence = np.array(img_arr_in_sequence)
pad_img_arr_in_sequence.shape

(10, 10, 5, 10, 25, 58, 3)

In [10]:
P, N, I, F, H, W, C = pad_img_arr_in_sequence.shape
X = np.reshape(pad_img_arr_in_sequence,(P*N*I*F,H,W,C))

X_normalized = X/255
X_normalized_t = np.transpose(X_normalized,(0,3,1,2))
X_normalized_t.shape

(5000, 3, 25, 58)

In [11]:
# Create ResNet34 model
resnet = resnet34_v2(pretrained=True)
resnet.hybridize()
resnet.collect_params().reset_ctx(mx.cpu())

# Create input tensor
inputs = X_normalized_t

# Reshape input tensor to have a batch size of 5000
x = np.reshape(inputs, (5000, 3, 25, 58))

# Convert to MXNet NDArray
x = mx.nd.array(x)

# Pass input through ResNet34 model
features_resnet = resnet(x)
print(features_resnet.shape)

resnet_feat = features_resnet.asnumpy()
resnet_feat = np.reshape(resnet_feat,(P*N*I,F,features_resnet.shape[-1]))
print(resnet_feat.shape)

(5000, 1000)
(500, 10, 1000)


In [12]:
X_normalized.shape

(5000, 25, 58, 3)

In [13]:

# Load the pre-trained VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False)

# Remove the last layer of the VGG16 model
features_model = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('block5_conv3').output)

# Reshape input data
input_shape = (25, 58, 3)
X = tf.keras.Input(shape=input_shape)
Y = tf.keras.layers.Reshape((25, 58, 3))(X)

# Pass input through VGG16 model
features = features_model(Y)

# Create model with inputs and outputs
model = tf.keras.Model(inputs=X, outputs=features)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Generate fake data for testing
x = X_normalized

features_vgg16 = model.predict(x)
print(features_vgg16.shape)


157/157 [==============================] - 9s 10ms/step
(5000, 1, 3, 512)


In [14]:
B,C,H,W = features_vgg16.shape
feat_vgg16 = np.reshape(features_vgg16,(B,C*H*W))
print(feat_vgg16.shape)
vgg16_feat = np.reshape(feat_vgg16,(P*N*I,F,feat_vgg16.shape[-1]))
print(vgg16_feat.shape)

(5000, 1536)
(500, 10, 1536)


In [15]:
target_words = []
for i in range(len(pad_img_arr_in_sequence)):
  for j in range(len(pad_img_arr_in_sequence[i])):
    for k in range(len(pad_img_arr_in_sequence[i][j])):
      target_words.append(j)

len(target_words)

500

In [ ]:
np.unique(np.array(target_words))

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

##LSTM with resnet and vgg16 features

In [16]:
X_train, X_test, y_train, y_test = train_test_split(resnet_feat, np.array(target_words), test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Training label shape:", y_train.shape)
print("Testing label shape:", y_test.shape)

Training data shape: (400, 10, 1000)
Testing data shape: (100, 10, 1000)
Training label shape: (400,)
Testing label shape: (100,)


In [ ]:
# Define model architecture
model = Sequential()
model.add(LSTM(units=128, input_shape=(10, 1000)))
model.add(Dense(units=10, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Split data into train and test sets

train_y = to_categorical(y_train, num_classes=10)
test_y = to_categorical(y_test, num_classes=10)

# Train model with input data of shape (400, 10, 1000) and validate on test set
model.fit(X_train, train_y , epochs=50, validation_data=(X_test, test_y), batch_size=64)

# Predict using the trained model
predictions = model.predict(X_test)
print(predictions.shape)

Epoch 1/50
7/7 [==============================] - 5s 118ms/step - loss: 2.3963 - accuracy: 0.1375 - val_loss: 2.3122 - val_accuracy: 0.1200
Epoch 2/50
7/7 [==============================] - 0s 15ms/step - loss: 2.1516 - accuracy: 0.2525 - val_loss: 2.2577 - val_accuracy: 0.2000
Epoch 3/50
7/7 [==============================] - 0s 16ms/step - loss: 2.0502 - accuracy: 0.2750 - val_loss: 2.2205 - val_accuracy: 0.1100
Epoch 4/50
7/7 [==============================] - 0s 15ms/step - loss: 1.9912 - accuracy: 0.3000 - val_loss: 2.2409 - val_accuracy: 0.1700
Epoch 5/50
7/7 [==============================] - 0s 15ms/step - loss: 1.9355 - accuracy: 0.3400 - val_loss: 2.1932 - val_accuracy: 0.2500
Epoch 6/50
7/7 [==============================] - 0s 20ms/step - loss: 1.8911 - accuracy: 0.3825 - val_loss: 2.1642 - val_accuracy: 0.2600
Epoch 7/50
7/7 [==============================] - 0s 16ms/step - loss: 1.8396 - accuracy: 0.4175 - val_loss: 2.1618 - val_accuracy: 0.1900
Epoch 8/50
7/7 [==========

In [17]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(vgg16_feat, np.array(target_words), test_size=0.2, random_state=42)

print("Training data shape:", X_train1.shape)
print("Testing data shape:", X_test1.shape)
print("Training label shape:", y_train1.shape)
print("Testing label shape:", y_test1.shape)

Training data shape: (400, 10, 1536)
Testing data shape: (100, 10, 1536)
Training label shape: (400,)
Testing label shape: (100,)


In [ ]:
# Define model architecture
model = Sequential()
model.add(LSTM(units=128, input_shape=(10, 1536)))
model.add(Dense(units=10, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Split data into train and test sets

train_y1 = to_categorical(y_train1, num_classes=10)
test_y1 = to_categorical(y_test1, num_classes=10)

# Train model with input data of shape (400, 10, 1000) and validate on test set
model.fit(X_train1, train_y1 , epochs=50, validation_data=(X_test1, test_y1), batch_size=32)

# Predict using the trained model
predictions = model.predict(X_test1)
print(predictions.shape)

Epoch 1/50
13/13 [==============================] - 3s 46ms/step - loss: 2.3927 - accuracy: 0.0950 - val_loss: 2.3320 - val_accuracy: 0.0900
Epoch 2/50
13/13 [==============================] - 0s 9ms/step - loss: 2.2803 - accuracy: 0.1500 - val_loss: 2.2956 - val_accuracy: 0.0700
Epoch 3/50
13/13 [==============================] - 0s 9ms/step - loss: 2.2631 - accuracy: 0.1350 - val_loss: 2.2684 - val_accuracy: 0.2100
Epoch 4/50
13/13 [==============================] - 0s 8ms/step - loss: 2.2199 - accuracy: 0.1450 - val_loss: 2.2798 - val_accuracy: 0.1400
Epoch 5/50
13/13 [==============================] - 0s 9ms/step - loss: 2.2128 - accuracy: 0.1900 - val_loss: 2.2392 - val_accuracy: 0.1700
Epoch 6/50
13/13 [==============================] - 0s 8ms/step - loss: 2.1890 - accuracy: 0.1975 - val_loss: 2.2596 - val_accuracy: 0.1400
Epoch 7/50
13/13 [==============================] - 0s 10ms/step - loss: 2.1452 - accuracy: 0.2175 - val_loss: 2.1840 - val_accuracy: 0.2100
Epoch 8/50
13/13 [

##Attention Models with resnet and Vgg16

In [ ]:
# Set the input shape
input_shape = (10, 1000)

# Define the attention-based LSTM model
inputs = Input(shape=input_shape)
lstm_out = LSTM(128, return_sequences=True)(inputs)
attention_out = Attention()([lstm_out, lstm_out])
flatten_out = Flatten()(attention_out)
dense_out = Dense(10, activation='softmax')(flatten_out)
model = Model(inputs=inputs, outputs=dense_out)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Split data into train and test sets
train_y = to_categorical(y_train, num_classes=10)
test_y = to_categorical(y_test, num_classes=10)

# Train model with input data of shape (400, 10, 1000) and validate on test set
model.fit(X_train, train_y , epochs=50, validation_data=(X_test, test_y), batch_size=32)

# Predict using the trained model
predictions_resnet_attention = model.predict(X_test)
print(predictions_resnet_attention.shape)

Epoch 1/50
13/13 [==============================] - 3s 48ms/step - loss: 2.3937 - accuracy: 0.1125 - val_loss: 2.3096 - val_accuracy: 0.1100
Epoch 2/50
13/13 [==============================] - 0s 10ms/step - loss: 2.1336 - accuracy: 0.2550 - val_loss: 2.3316 - val_accuracy: 0.1100
Epoch 3/50
13/13 [==============================] - 0s 10ms/step - loss: 2.0129 - accuracy: 0.2925 - val_loss: 2.1683 - val_accuracy: 0.1600
Epoch 4/50
13/13 [==============================] - 0s 9ms/step - loss: 1.8651 - accuracy: 0.3750 - val_loss: 2.1151 - val_accuracy: 0.1900
Epoch 5/50
13/13 [==============================] - 0s 10ms/step - loss: 1.7254 - accuracy: 0.4375 - val_loss: 2.0034 - val_accuracy: 0.2400
Epoch 6/50
13/13 [==============================] - 0s 9ms/step - loss: 1.5581 - accuracy: 0.5150 - val_loss: 1.9184 - val_accuracy: 0.2900
Epoch 7/50
13/13 [==============================] - 0s 9ms/step - loss: 1.4043 - accuracy: 0.5725 - val_loss: 1.8832 - val_accuracy: 0.3300
Epoch 8/50
13/13

In [ ]:
# Set the input shape
input_shape = (10, 1536)

# Define the attention-based LSTM model
inputs = Input(shape=input_shape)
lstm_out = LSTM(128, return_sequences=True)(inputs)
attention_out = Attention()([lstm_out, lstm_out])
flatten_out = Flatten()(attention_out)
dense_out = Dense(10, activation='softmax')(flatten_out)
model = Model(inputs=inputs, outputs=dense_out)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Split data into train and test sets
train_y1 = to_categorical(y_train1, num_classes=10)
test_y1 = to_categorical(y_test1, num_classes=10)

# Train model with input data of shape (400, 10, 1000) and validate on test set
model.fit(X_train1, train_y1 , epochs=50, validation_data=(X_test1, test_y1), batch_size=32)

# Predict using the trained model
predictions_vgg16_attention = model.predict(X_test1)
print(predictions_vgg16_attention.shape)

Epoch 1/50
13/13 [==============================] - 6s 48ms/step - loss: 2.4540 - accuracy: 0.0975 - val_loss: 2.3500 - val_accuracy: 0.0900
Epoch 2/50
13/13 [==============================] - 0s 10ms/step - loss: 2.3052 - accuracy: 0.1200 - val_loss: 2.3256 - val_accuracy: 0.0800
Epoch 3/50
13/13 [==============================] - 0s 10ms/step - loss: 2.2823 - accuracy: 0.1300 - val_loss: 2.3251 - val_accuracy: 0.1100
Epoch 4/50
13/13 [==============================] - 0s 9ms/step - loss: 2.2794 - accuracy: 0.1225 - val_loss: 2.3127 - val_accuracy: 0.1200
Epoch 5/50
13/13 [==============================] - 0s 10ms/step - loss: 2.2339 - accuracy: 0.1925 - val_loss: 2.3404 - val_accuracy: 0.0900
Epoch 6/50
13/13 [==============================] - 0s 9ms/step - loss: 2.2061 - accuracy: 0.1925 - val_loss: 2.3690 - val_accuracy: 0.0900
Epoch 7/50
13/13 [==============================] - 0s 11ms/step - loss: 2.2067 - accuracy: 0.1825 - val_loss: 2.1856 - val_accuracy: 0.2200
Epoch 8/50
13/1

In [182]:
class Traditional_Positional_Encoding(Layer):
    def __init__(self, **kwargs):
        super(Traditional_Positional_Encoding, self).__init__(**kwargs)
        self.seq_len = 10

    def call(self, inputs):
      #inputs = np.array(inputs)
      input_dim = inputs.shape[-1]
      seq_len = self.seq_len
      max_len=100.0
      k = np.arange(0,input_dim,dtype=float)[None,:]
      pos = np.arange(0,seq_len, dtype=float)[:,None]
        
      i = k//2
      pe = pos/(max_len**(2*i/input_dim))
      pe[:,0::2] = np.sin(pe[:,0::2])
      pe[:,1::2] = np.cos(pe[:,1::2])
      pe = tf.convert_to_tensor(pe)
      pe = tf.expand_dims(pe, axis=0)
      pe = tf.tile(pe, [None, 1, 1])
      return pe

class Learned_Positional_Encoding(Layer):
    def __init__(self, **kwargs):
        super(Learned_Positional_Encoding, self).__init__(**kwargs)

    def call(self, inputs):
      input_dim = inputs.shape[-1]
      seq_len, batch_size = inputs.shape[1], inputs.shape[0]

      pe = inputs * tf.math.sqrt(tf.cast(input_dim, tf.float32))
      pe = Lambda(lambda i: tf.expand_dims(i, axis=-1))(pe)
      pe = Conv1D(filters=64, kernel_size=1, activation='relu')(pe)
      pe = Conv1D(filters=64, kernel_size=1, activation='linear')(pe)
      pe = Dense(units=1, activation=None)(pe)
      pe = tf.reshape(pe,[batch_size,seq_len,input_dim])
      #print(pe.shape)
      return pe

In [197]:
def transformer_block(inputs, num_heads, ff_dim, dropout, name):
  attention_out = MultiHeadAttention(num_heads = num_heads, key_dim=128, name=name+'_multi_head')(inputs,inputs)
  #attention_out = Dense(inputs.shape[-1], name=name+'_attn_dense')(attention_out)
  dropout1 = Dropout(dropout)(attention_out)
  out1 = LayerNormalization()(inputs+dropout1)
  ffn = Sequential([
      Dense(ff_dim,activation='relu'),
      Dense(inputs.shape[2])], name=name+"_ffn")
  ffn_out = ffn(out1)
  dropout1 = Dropout(dropout)(ffn_out)
  final_out = LayerNormalization()(out1+dropout1)
  return final_out

def transformer_model(input, num_transformers, num_heads, ff_dim, dropout, pos_encod_type):
  if pos_encod_type=='traditional':
    pos_enc = Traditional_Positional_Encoding()(input)
  elif pos_encod_type=='learned':
    pos_enc = Learned_Positional_Encoding()(input)
  else:
    pos_enc = 0

  x = pos_enc + input
  for i in range(num_transformers):
    x = transformer_block(x,num_heads=num_heads,ff_dim =ff_dim, dropout=dropout,name="Transformer_"+str(i+1))
  x = GlobalAveragePooling1D()(x)
  out = Dense(10,activation='softmax')(x)
  model = Model(inputs=input,outputs=out)
  return model

In [199]:
input_shape = (10, 1000)
input = tf.keras.layers.Input(shape=input_shape)
model = transformer_model(input, num_transformers=10, num_heads=32, ff_dim=512, dropout=0.1, pos_encod_type='None')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_y = to_categorical(y_train, num_classes=10)
test_y = to_categorical(y_test, num_classes=10)

# Train model with input data of shape (400, 10, 1000) and validate on test set
model.fit(X_train, train_y , epochs=50, validation_data=(X_test, test_y), batch_size=32)#, callbacks=[EarlyStopping(patience=3)])

preditions_resnet_transformer = model.predict(X_test)
preditions_resnet_transformer.shape

Epoch 1/50
13/13 [==============================] - 40s 318ms/step - loss: 8.1903 - accuracy: 0.1350 - val_loss: 4.7773 - val_accuracy: 0.1400
Epoch 2/50
13/13 [==============================] - 2s 176ms/step - loss: 3.8885 - accuracy: 0.0775 - val_loss: 3.2436 - val_accuracy: 0.0500
Epoch 3/50
13/13 [==============================] - 2s 166ms/step - loss: 2.7894 - accuracy: 0.0875 - val_loss: 2.6331 - val_accuracy: 0.1300
Epoch 4/50
13/13 [==============================] - 2s 156ms/step - loss: 2.5211 - accuracy: 0.1150 - val_loss: 2.5496 - val_accuracy: 0.1000
Epoch 5/50
13/13 [==============================] - 2s 170ms/step - loss: 2.5125 - accuracy: 0.0900 - val_loss: 2.4248 - val_accuracy: 0.1400
Epoch 6/50
13/13 [==============================] - 2s 168ms/step - loss: 2.4668 - accuracy: 0.0850 - val_loss: 2.6080 - val_accuracy: 0.0800
Epoch 7/50
13/13 [==============================] - 2s 169ms/step - loss: 2.4748 - accuracy: 0.0875 - val_loss: 2.5326 - val_accuracy: 0.0900
Epoch

(100, 10)

In [193]:
input_shape = (10, 1000)
input = tf.keras.layers.Input(shape=input_shape)
model = transformer_model(input, num_transformers=25, num_heads=8, ff_dim=512, dropout=0.1, pos_encod_type='None')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_y = to_categorical(y_train, num_classes=10)
test_y = to_categorical(y_test, num_classes=10)

# Train model with input data of shape (400, 10, 1000) and validate on test set
model.fit(X_train, train_y , epochs=50, validation_data=(X_test, test_y), batch_size=32)#, callbacks=[EarlyStopping(patience=3)])

preditions_resnet_transformer = model.predict(X_test)
preditions_resnet_transformer.shape

Epoch 1/50
13/13 [==============================] - 108s 820ms/step - loss: 7.4184 - accuracy: 0.0950 - val_loss: 4.3985 - val_accuracy: 0.1400
Epoch 2/50
13/13 [==============================] - 6s 434ms/step - loss: 3.2749 - accuracy: 0.1025 - val_loss: 2.9539 - val_accuracy: 0.0500
Epoch 3/50
13/13 [==============================] - 6s 450ms/step - loss: 2.7635 - accuracy: 0.1125 - val_loss: 2.5129 - val_accuracy: 0.1000
Epoch 4/50
13/13 [==============================] - 6s 436ms/step - loss: 2.4969 - accuracy: 0.0775 - val_loss: 2.5347 - val_accuracy: 0.0800
Epoch 5/50
13/13 [==============================] - 6s 449ms/step - loss: 2.4479 - accuracy: 0.0875 - val_loss: 2.4745 - val_accuracy: 0.0500
Epoch 6/50
13/13 [==============================] - 6s 445ms/step - loss: 2.4393 - accuracy: 0.0925 - val_loss: 2.6104 - val_accuracy: 0.0500
Epoch 7/50
13/13 [==============================] - 6s 435ms/step - loss: 2.4654 - accuracy: 0.0950 - val_loss: 2.5160 - val_accuracy: 0.0900
Epoc

(100, 10)

In [187]:
input_shape = (10, 1000)
input = tf.keras.layers.Input(shape=input_shape)
model = transformer_model(input, num_transformers=3, num_heads=8, ff_dim=512, dropout=0.1, pos_encod_type='learned')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_y = to_categorical(y_train, num_classes=10)
test_y = to_categorical(y_test, num_classes=10)

# Train model with input data of shape (400, 10, 1000) and validate on test set
model.fit(X_train, train_y , epochs=50, validation_data=(X_test, test_y), batch_size=32)#, callbacks=[EarlyStopping(patience=3)])

preditions_resnet_transformer = model.predict(X_test)
preditions_resnet_transformer.shape

(None, 10, 1000)


TypeError: ignored